In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import psycopg2
import os
import sqlalchemy
import sys
# from sqlalchemy import create_engine
import pipeline_plan_functions.utils.pipe_db_handler as dbh
from python_utils.utils.logger import logger
logger.setLevel('DEBUG')
import pipeline_plan_functions.utils.data_handler as dh
import json
# from pulp import *
# import itertools
# import scheduling_functions.optimisation as opt
# import scheduling_functions.schedule_scenario as ssf
from scheduling_functions import controller
from scheduling_functions import cleanup
# from scheduling_functions import optimisation as opt
# import cvxpy as cp
# from cvxopt.modeling import variable, op, max, sum

In [3]:
cnx = dbh.create_alch_engine()
sql_query = "select * from t_allocation where run_id=203"
alloc = pd.read_sql_query(sql_query, con=cnx)

In [6]:
alloc['allocation_id'].values

array([671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683,
       684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696,
       697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709,
       710, 711, 712, 713, 714, 715, 718, 721, 722, 723, 724, 728, 731,
       732, 733, 738, 739, 743], dtype=int64)

In [1]:
string_list = """452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493"""

In [8]:
import numpy as np
np.array(string_list.split('\n')).astype(int)

array([452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464,
       465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477,
       478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490,
       491, 492, 493])

In [39]:
# connection, cur = dbh.database_connection('test')
sql_query = (
    f"""SELECT allocation_id FROM t_allocation
    WHERE site_id IN (278, 279, 280, 281, 282)
    ORDER BY allocation_id""")
cur.execute(sql_query)
connection.commit()
values = cur.fetchall()
allocations = str([v[0] for v in values])

In [2]:
connection, cur = dbh.database_connection('test')
sql_query = (
    f"""SELECT allocation_id, vehicle2 FROM t_allocation
    WHERE allocation_id>=126 AND allocation_id<=176
    ORDER BY allocation_id""")
cur.execute(sql_query)
connection.commit()
values = cur.fetchall()
allocations = [pair[0] for pair in values if pair[1] != 5]
for x in [129, 130, 167]:
    allocations.remove(x)

In [36]:
cnx = dbh.create_alch_engine()
last_id = 168569
# table = 't_route_master'
table = 't_route_formatted'
query = f"SELECT * FROM {table} WHERE site_id_start = 282 AND source=201 ORDER BY route_id"
routes = pd.read_sql_query(query, cnx)
routes['route_id'] = last_id + 1 + routes.index
dbh.upload_table(routes, table)
cnx.dispose()

2022-05-06 13:15:14|DEBUG|db_handler.py(62)|upload_table|Uploaded to t_route_formatted


In [31]:
routes.head()

,client_id,site_id_start,departure_time,arrival_time,vehicle_id,distance_miles,route_id,payload,number_crates,number_order,driving_time,site_id_end,source
0,2,279,2021-09-10 06:31:00,2021-09-10 11:43:00,None,20.0,168467,381,-1,20,3.39,279,201
1,2,279,2021-09-10 06:24:00,2021-09-10 11:26:00,None,20.0,168468,301,-1,17,2.97,279,201
2,2,279,2021-09-10 06:49:00,2021-09-10 12:34:00,None,9.0,168469,318,-1,25,3.63,279,201
3,2,279,2021-09-10 07:00:00,2021-09-10 07:33:00,None,4.0,168470,49,-1,1,0.22,279,201
4,2,279,2021-09-10 07:00:00,2021-09-10 12:30:00,None,45.0,168471,229,-1,20,3.96,279,201


In [2]:
# connection, cur = dbh.database_connection('test')
# cnx = dbh.create_alch_engine()

In [4]:
TIME_FRACT = 0.5
CHARGER_EFF = 0.9  # FIXME put this in the database
BATTERY_FACTOR = 0
ASC_XUSE = 0.9
scenario = 5

connection, cur = dbh.database_connection('test')
# Get run input parameters
cnx = dbh.create_alch_engine()
params = ssf.get_scheduling_inputs(scenario, connection, cur, cnx)
params = ssf.get_site_data(params, connection, cur)
params['asc_kw'] = 120  #############
# Get routes
routes = ssf.get_allocated_routes(params['allocation_id'], cnx)
routes = ssf.get_route_data(routes, params, cnx)
# # Get list of dates
dates = ssf.list_dates(routes['date'])
# # Get time periods (T time periods)
times = ssf.create_time_periods(dates, params)
ndays = (times[-1] - times[0] + dt.timedelta(hours=TIME_FRACT)).days
# Get site load
capacity = ssf.get_site_capacity(params['site_id'], times,
                                params['smart_charging'],
                                params['asc_kw'], cnx)
# Get electricity price
electricity = ssf.get_tariff(params['distribution_id'], times,
                            params['smart_charging'], cnx)
# Get vehicle list (N vehicles)
vehicles = sorted(routes['allocated_vehicle_id'].unique())
# Get vehicle availability, energy use matrix (T*N) and session matrix
available, evuse, session_matrix = ssf.vehicle_matrices(
    vehicles, routes, times)
# Get vehicle spec vectors
battery_cap, charger_rate_ac, charger_rate_dc = ssf.vehicle_spec_vectors(
    routes, vehicles, connection, cur)
charger_efficiency = np.diag(np.full(int(24/TIME_FRACT), CHARGER_EFF))
# # Make relative charge and required energy vectors
req_energy_level = -battery_cap  # FIXME
rel_charge = np.zeros(len(vehicles))
vehicle_vectors = [rel_charge, req_energy_level, battery_cap, vehicles,
                    charger_rate_ac, charger_rate_dc]
site_vectors = [electricity, capacity, times]
# iterate over each day and filter inputs
breach_days = []
magic_days = []
breaches = 0
output_kwh = 0
excess_costs = 0
for day in range(ndays)[:]:
    start = int(day*24/TIME_FRACT)
    end = int((day+1)*24/TIME_FRACT)
    matrices = ssf.day_matrices(available, evuse, session_matrix,
                            charger_efficiency, start, end)
    day_vectors = ssf.day_site_vectors(site_vectors, start, end)
    # run linear optimiser for each day
    final_soc, opt_level, evout = opt.linear_optimiser_V9(
        matrices, day_vectors, vehicle_vectors, params)
    if opt_level == 'breach':
        breach_days.append(day)
    elif opt_level == 'magic':
        magic_days.append(day)
    # Export each day
    cleanup.export_charge_schedule(evout, params, vehicles,
                                    times[start:end], cnx)
    breaches += cleanup.calculate_breaches(evout, day_vectors[1])
    output_kwh += evout.sum()
    excess_costs += ssf.excess_capacity_cost(
        evout, day_vectors[1], params['distribution_id'], connection, cur)
    # update relative energy vector
    vehicle_vectors[0] = final_soc
    print(excess_costs)
n_breach_days = len(breach_days)
n_magic_days = len(magic_days)
cleanup.update_scenarios(breaches, n_breach_days, n_magic_days,
                            output_kwh, excess_costs, scenario, connection, cur)

2022-04-07 15:54:15|DEBUG|data_handler.py(37)|get_inputs|read inputs for run 22
2022-04-07 15:54:19|INFO|optimisation.py(125)|linear_optimiser_V9|Optimisation succesfully run in normal mode
2022-04-07 15:54:19|DEBUG|cleanup.py(37)|export_charge_schedule|Updated schedule for scenario 5in t_charge_demand


Long-step dual simplex will be used
5.481329675863631e-16


2022-04-07 15:54:20|WARNING|optimisation.py(121)|linear_optimiser_V9|=================BREACH========================
2022-04-07 15:54:21|INFO|optimisation.py(330)|linear_optimiser_breach|Optimisation status (breach mode): optimal
2022-04-07 15:54:21|DEBUG|cleanup.py(37)|export_charge_schedule|Updated schedule for scenario 5in t_charge_demand


Long-step dual simplex will be used
22.21714285714286


2022-04-07 15:54:22|WARNING|optimisation.py(121)|linear_optimiser_V9|=================BREACH========================
2022-04-07 15:54:23|INFO|optimisation.py(330)|linear_optimiser_breach|Optimisation status (breach mode): optimal
2022-04-07 15:54:23|DEBUG|cleanup.py(37)|export_charge_schedule|Updated schedule for scenario 5in t_charge_demand


...

Long-step dual simplex will be used
215.5506278597143


2022-04-07 15:54:54|INFO|optimisation.py(125)|linear_optimiser_V9|Optimisation succesfully run in normal mode
2022-04-07 15:54:54|DEBUG|cleanup.py(37)|export_charge_schedule|Updated schedule for scenario 5in t_charge_demand


Long-step dual simplex will be used
215.5506278597143


2022-04-07 15:54:56|INFO|optimisation.py(125)|linear_optimiser_V9|Optimisation succesfully run in normal mode
2022-04-07 15:54:56|DEBUG|cleanup.py(37)|export_charge_schedule|Updated schedule for scenario 5in t_charge_demand
2022-04-07 15:54:56|DEBUG|cleanup.py(68)|update_scenarios|Updated t_charging_scnearios with QA


Long-step dual simplex will be used
215.5506278597143


In [37]:
# cnx.dispose()
cur.close()
connection.close()
# cnx.close()

In [ ]:
connection.commit()

## Dummy data

In [ ]:
# cnx = dbh.create_alch_engine()
# query = "SELECT * FROM t_run_charging WHERE run_id=21"
# run_charging = pd.read_sql(query, con=cnx)
run_charging['run_id'] = 22
run_charging.to_sql('t_run_charging', con=cnx, if_exists='append', index=None)

In [ ]:
# DELETE data from scheduler
sql_query = "DELETE FROM t_charge_demand WHERE scenario_id=6"
cur.execute(sql_query)
connection.commit()

In [ ]:
# Charging profiles

startdt = dt.datetime(2021, 10, 17, 7)
enddt = dt.datetime(2021, 11, 7, 7)
tps = pd.date_range(start=startdt, end=enddt, freq='0.5H')
vehicles = np.arange(1, 27)
df = pd.DataFrame(index=pd.MultiIndex.from_product([tps, vehicles],
                                                   names=['datetime', 'allocated_vehicle_id']),
                  columns=['scenario_id', 'power_demand_kw'])
N = len(df)
df['scenario_id'] = 6
df['power_demand_kw'] = 22*np.random.random(size=N) * np.random.choice([0,1], N)
df.to_sql('t_charge_demand', con=cnx, if_exists='append')

In [ ]:
## Meter data
meter = pd.DataFrame()
meter['datetime'] = meter_times
meter['site_id'] = 24
meter['clean_load'] = np.random.random(len(meter))*350
meter['client_id'] = 2
meter.to_sql('t_site_load', con=cnx, if_exists='append', index=False)